In [1]:
#@title Depedencies
!pip install -q pinecone-client==2.2.1 openai==0.27.4 gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...

In [2]:
# @title Code and keys
import openai
import pinecone

openai.api_key = "GET YOUR KEY FROM https://beta.openai.com/account/api-keys"
embed_model = "text-embedding-ada-002"
index_name = "youtube-transcriptions"
pinecone.init(
    api_key="GET YOUR KEY FROM https://app.pinecone.io/",
    environment="us-west1-gcp",  # may be different, check at app.pinecone.io
)
index = pinecone.Index(index_name)

/usr/local/lib/python3.9/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# @title Examples and code
limit = 1e19


def retrieve(query, k=5):
    embed_model = "text-embedding-ada-002"
    res = openai.Embedding.create(input=[query], engine=embed_model)

    # retrieve from Pinecone
    xq = res["data"][0]["embedding"]

    # get relevant contexts
    res = index.query(xq, top_k=k, include_metadata=True)
    contexts = [x["metadata"]["text"] for x in res["matches"]]
    prompt_start = "Answer the question based on the context below.\n\n" + "Context:\n"
    prompt_end = (
        # f"\n\nQuestion: {query}\nAnswer:"
        f"\n\n"
    )
    # append contexts until hitting limit
    # print(len(contexts))
    for i in range(1, len(contexts)):
        # print(len("\n\n---\n\n".join(contexts[:i])))
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = prompt_start + "\n\n---\n\n".join(contexts[: i - 1]) + prompt_end
            break
        elif i == len(contexts) - 1:
            prompt = prompt_start + "\n\n---\n\n".join(contexts) + prompt_end
    return prompt
    # return prompt


def answer(context, query):
    primer = f"""You are Q&A bot. A highly intelligent system that answers 
  user questions based on the information provided by videos of kubeflow, katib, that may have errors on the transcriptions. You can use your inner knowledge,
  but consider more the information provided. Put emphasis on the transcriptions provided
  """

    res = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": primer},
            {"role": "user", "content": context},
            {"role": "user", "content": query},
        ],
    )
    return res["choices"][0]["message"]["content"]

In [ ]:
import gradio as gr


def question_answer(question):
    k = 6
    try:
        context = retrieve(question, k=k)
        answer_txt = answer(context, question)
    except Exception:
        try:
            context = retrieve(question, k=k - 1)
            answer_txt = answer(context, question)
        except Exception:
            context = retrieve(question, k=k - 3)
            answer_txt = answer(context, question)

    return answer_txt


app = gr.Interface(
    fn=question_answer,
    inputs=["text"],
    outputs=["markdown"],
    title="Kubeflow Tribal Knowledge",
    css=".gradio-container {background-color: Ghost White}",
)

# Set the background color to light blue using CSS styles
# app.interface_style = {"body": {"background-color": }}
app.launch(share=True, debug=True)